![logo_universidad](https://bloqueneon.uniandes.edu.co//content/enforced/171175-202310_ISIS3301_02/morado/assets/img/banner.png?_&d2lSessionVal=d1HNZaZihV1jOwJNVtWDIWG7y&ou=171175)

## Laboratorio 1 - Clustering

El objetivo de este notebook es realizar el analisis del caso BiciAlpes, para el cual se tiene un conjunto de datos que contiene información de los accidentes ocurridos involucrando bicicletas en la ciudad de Bogotá. El objetivo es realizar un análisis exploratorio de los datos para identificar patrones y relaciones entre las variables, y así poder responder las preguntas de negocio planteadas. Adicionalmente, se espera usar 3 algoritmos de agrupamiento para identificar grupos de accidentes que se puedan utilizar para responder las preguntas de negocio.

Por medio de 3 iteraciones, se espera que el grupo desarrolle un modelo de agrupamiento que permita responder las preguntas de negocio. En cada iteración se espera que nuestro equipo de trabajo realice los siguientes pasos:

* **Iteración1.** Analisis exploratorio de los datos y creación de un primer modelo de agrupamiento.
* **Iteración2.** Segundo modelo con mejoras a nivel de preparación de datos y configuración de hiper-parámetros
* **Iteración3.** Modelo final que incluye mejoras identificadas al revisar mejor la literatura y la documentación de los algoritmos de agrupamiento.

### Descripción de negocio

BiciAlpes es una organización que presta un servicio de alquiler de bicicletas, solidario y con muchos beneficios para los usuarios. Su objetivo es incentivar planes de movilidad sostenible en zonas urbanas, que contribuyan a reducir la emisión de gases de efecto invernadero que afectan la temperatura del planeta. En el último año, han detectado con preocupación que el uso de la bicicleta se está viendo obstaculizado por la falta de información sobre la seguridad en las vías. Piensa que proporcionar a los ciclistas información sobre las rutas más seguras podría contribuir a aumentar la práctica de esta actividad. Así, decide emprender un estudio para conocer cuáles son los factores que más impactan en los accidentes viales que involucran ciclistas. Este conocimiento, además, podría ayudar a las autoridades y planificadores urbanos en la implementación de mecanismos que reduzcan la ocurrencia de accidentes, así como otros planes de movilidad sostenible. Para realizar este estudio BiciAlpes han recopilado datos, de fuentes abiertas de la Alcaldía, sobre accidentes en los cuales se han visto involucrados ciclistas.

BiciAlpes los ha contactado para realizar este proceso de caracterización e identificación de patrones que puedan ser utilizados para entender los factores que indicen en la severidad de los accidentes, con el fin de trazar mapas de seguridad vial para los ciclistas.  Los datos, al igual que el diccionario de datos, los encuentra disponibles en este directorio: [datos](./datos/Datos_BiciAlpes.csv)

## Referencias

1. [Datos a utilizar: Datos_BiciAlpes.csv](./datos/Datos_BiciAlpes.csv)
2. [Diccionario de datos: Diccionario_BiciAlpes.csv](./datos/Diccionario_BiciAlpes.xlsx)
3. [Documentación de los algoritmos de agrupamiento](https://scikit-learn.org/stable/modules/clustering.html)
4. [Documentación de la librería de visualización de datos](https://seaborn.pydata.org/)
5. [Informacion del negocio](./Laboratorio1_enunciado.md)

## 1. Carga de librerias necesarias

In [21]:
#Se cargan las librerias necesarias para la implementacion
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
# Este comando es requerido para que las visualizaciones se muestren en este notebook
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture._gaussian_mixture import GaussianMixture

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

#Scipy para aglomeracion
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster

In [ ]:
pd.set_option('display.max_columns', None) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar

## Exploración y perfilamiento de los datos

## 2. Carga de datos

In [23]:
# Se cargan los datos necesarios para la implementacion
df_accidents = pd.read_csv('./datos/Datos_BiciAlpes.csv', sep=';', encoding = 'latin-1', index_col=0)

In [24]:
# Se muestran las dimensiones del dataframe
df_accidents.shape

(5338, 14)

In [25]:
# Tomamos una muestra aleatoria de los datos para observar su información
df_accidents.sample(5)

,Number_of_Casualties,Day_of_Week,Road_Type,Speed_limit,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Vehicle_Type,Did_Police_Officer_Attend_Scene_of_Accident,Junction_Detail,Number_of_Vehicles,Accident_Severity,Unnamed: 14
Time,,,,,,,,,,,,,,
Mañana,1,Fin de semana,6,30.0,1,1,1,1,bike,1,3,1,3,NaN
Tarde,1,Día laboral,6,40.0,1,1,1,1,bike,1,0,1,3,NaN
Noche,1,Día laboral,6,60.0,1,1,1,2,bike,1,0,1,2,NaN
Mañana,1,Día laboral,6,60.0,1,1,1,2,bike,1,3,1,2,NaN
Noche,1,Día laboral,6,30.0,4,1,1,1,bike,1,0,1,3,NaN


Se puede observar que existe una columna llamada "Unnamed: 14" la cual cuenta con valores nulos en su totalidad. Esta no representa información relevante para el análisis, por lo que se procede a continuar con el perfilamineto para ver su comportamiento.

## 3. Descripción de los datos

In [27]:
# Se muestran los tipos de datos de cada columna
df_accidents.dtypes

Number_of_Casualties                             int64
Day_of_Week                                     object
Road_Type                                        int64
Speed_limit                                    float64
Light_Conditions                                 int64
Weather_Conditions                               int64
Road_Surface_Conditions                          int64
Urban_or_Rural_Area                              int64
Vehicle_Type                                    object
Did_Police_Officer_Attend_Scene_of_Accident      int64
Junction_Detail                                  int64
Number_of_Vehicles                               int64
Accident_Severity                                int64
Unnamed: 14                                    float64
dtype: object

### 3.1 Información de las variables númericas

In [32]:
# Extraemos las variables númericas mencionadas en el diccionario
df_accidents_numeric = df_accidents[['Number_of_Casualties', 'Speed_limit', 'Number_of_Vehicles']]

# Describimos las variables númericas
df_accidents_numeric.describe()

,Number_of_Casualties,Speed_limit,Number_of_Vehicles
count,5338.000000,5338.000000,5338.000000
mean,1.118022,33.522855,1.001124
std,0.330694,10.206576,0.033511
min,1.000000,15.000000,1.000000
25%,1.000000,30.000000,1.000000
50%,1.000000,30.000000,1.000000
75%,1.000000,30.000000,1.000000
max,4.000000,70.000000,2.000000


Para los resultados de las variables numericas que tenemos, se tiene la siguiente información:

* Para todas existe un numero de accidentes de 5338, lo cual es correcto ya que es el total de accidentes que tenemos en el dataset.

### Number_of_Casualties

* Para esta variable se tiene que el valor minimo es 1 y máximo es 4, lo cual nos indica que para cada accidente se tiene un numero de personas involucradas que va desde 1 hasta 4.

* El valor promedio es de 1.12 lo cual nos indica que en promedio para cada accidente se tiene una persona involucrada.

* En cuanto a los porcentajes de los datos, tenemos que para el 25%, 50% y 75% de los datos se tiene que el numero de personas involucradas es 1, lo cual nos indica que la mayoria de los accidentes se tiene una sola persona involucrada. Adicionalmente, esto nos indica que el caso que se muestra como maximo se encuentra fuera del 75% de los datos.

### Speed_limit

* Para esta variable se tiene que el valor minimo es 15 y máximo es 70, lo cual nos indica que para cada accidente se tiene un limite de velocidad que va desde 15 hasta 70.

* El valor promedio es de 33.52 lo cual nos indica que en promedio para cada accidente se tiene un limite de velocidad de 32.52.

* En cuanto a los porcentajes de los datos, tenemos que para el 25%, 50% y 75% de los datos se tiene que el limite de velocidad es 30, lo cual nos indica que la mayoria de los accidentes se tiene un limite de velocidad de 30. Adicionalmente, esto nos indica que el caso que se muestra como maximo se encuentra fuera del 75% de los datos.

### Number_of_Vehicles

* Para esta variable se tiene que el valor minimo es 1 y máximo es 2, lo cual nos indica que para cada accidente se tiene un numero de vehiculos involucrados que va desde 1 hasta 2.

* El valor promedio es de 1.001 lo cual nos indica que en promedio para cada accidente se tiene un numero de vehiculos involucrados de 1.001.

* En cuanto a los porcentajes de los datos, tenemos que para el 25%, 50% y 75% de los datos se tiene que el numero de vehiculos involucrados es 1, lo cual nos indica que la mayoria de los accidentes se tiene un solo vehiculo involucrado. Adicionalmente, esto nos indica que el caso que se muestra como maximo se encuentra fuera del 75% de los datos.

Teniendo en cuenta la información anterior, es posible encontrar una relación entre el numero de involucrados y el numero de vehiculos para asi poder determinar si es posible agrupar los datos en base a esta relación.

In [33]:
# Se encuentra la relacion entre las variables númericas (Number_of_Casualties y Number_of_Vehicles)
df_accidents["CasualtiesbyVehicle"] = df_accidents["Number_of_Casualties"]/df_accidents["Number_of_Vehicles"]

In [40]:
df_accidents["CasualtiesbyVehicle"].describe()

count    5338.000000
mean        1.116898
std         0.329391
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
Name: CasualtiesbyVehicle, dtype: float64

En base a esta información, tenemos que la media de accidentes por numero de vehiculo es 1.117 lo cual nos indica que en promedio para cada vehiculo involucrado se tiene un accidente. Adicionalmente, se puede asumir haciendo el analisis adecuado que en cada accidente el involucrado es el ciclista y el vehiculo es el perpertrador del accidente.

In [36]:
df_accidents[df_accidents["CasualtiesbyVehicle"] == 4]

,Number_of_Casualties,Day_of_Week,Road_Type,Speed_limit,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Vehicle_Type,Did_Police_Officer_Attend_Scene_of_Accident,Junction_Detail,Number_of_Vehicles,Accident_Severity,Unnamed: 14,CasualtiesbyVehicle
Time,,,,,,,,,,,,,,,
Tarde,4,Fin de semana,6,30.0,1,2,2,1,bike,1,3,1,3,NaN,4.0
Mañana,4,Día laboral,6,30.0,1,1,1,1,bike,1,6,1,3,NaN,4.0


### 3.2 Información de las variables categóricas

In [57]:
# Extraemos las variables númericas mencionadas en el diccionario
df_accidents_categorica = df_accidents[['Accident_Severity', 'Day_of_Week', 'Road_Type', 'Junction_Detail', 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Urban_or_Rural_Area', 'Vehicle_Type', 'Did_Police_Officer_Attend_Scene_of_Accident']]

In [55]:
# Describimos la variable categorica Accident_Severity
df_accidents_categorica['Accident_Severity'].value_counts()

3    3462
2    1781
1      95
Name: Accident_Severity, dtype: int64

De la severidad de los accidentes, se puede decir que la mayoria de los accidentes son leves, seguido de los accidentes serios y los accidentes mortales.

In [56]:
# Describimos las variable categorica Day_of_Week
df_accidents_categorica['Day_of_Week'].value_counts()

Día laboral      3832
Fin de semana    1487
Name: Day_of_Week, dtype: int64

De los accidentes podemos decir que la mayoria ocurren en un dia laboral y el resto los fines de semana.

In [58]:
# Describimos las variable categorica Road_Type
df_accidents_categorica['Road_Type'].value_counts()

6    4475
3     312
2     253
9     141
1     139
7      18
Name: Road_Type, dtype: int64

De los tipos de carretera tenemos que la mayoría de accidentes ocurren en carreteras con calzada seguido de carreteras autovia, carreteras de un solo sentido, de origen desconocido, rotondas y circunvalación.

In [59]:
# Describimos las variable categorica Junction_Detail
df_accidents_categorica['Junction_Detail'].value_counts()

 0    2763
 3    1555
 6     468
 1     199
 9     171
 7      68
 8      43
 2      34
-1      25
 5      12
Name: Junction_Detail, dtype: int64

En cuanto a las caracteristicas del cruce, tenemos que se presentaron mas accidentes donde no habían cruces y menos donde se contaba con una carretera de acceso.

In [60]:
# Describimos las variable categorica Light_Conditions
df_accidents_categorica['Light_Conditions'].value_counts()

1    4231
4     891
6     122
7      73
5      21
Name: Light_Conditions, dtype: int64

En cuanto a las condiciones de iluminación, la mayor cantidad de accidentes se presentó durante el día, y la menor cantidad con iluminación desconocida.

In [61]:
# Describimos las variable categorica Weather_Conditions
df_accidents_categorica['Weather_Conditions'].value_counts()

1    4621
2     321
9     191
8      80
4      59
5      46
3      12
7       7
6       1
Name: Weather_Conditions, dtype: int64

En cuanto a las condiciones climaticas, la cantidad de accidentes es mayor cuando este presenta un buen clima y vientos suaves y menor cuando el clima presenta nieve. 

In [62]:
# Describimos las variable categorica Road_Surface_Conditions
df_accidents_categorica['Road_Surface_Conditions'].value_counts()

 1    4260
 2     901
 4      87
-1      81
 3       8
 5       1
Name: Road_Surface_Conditions, dtype: int64

En cuanto a las condiciones de la superficie de la carretera, la mayor cantidad de accidentes se presentó en carreteras secas y la menor cantidad en carreteras inundadas.

In [63]:
# Describimos las variable categorica Urban_or_Rural_Area
df_accidents_categorica['Urban_or_Rural_Area'].value_counts()

1    4040
2    1298
Name: Urban_or_Rural_Area, dtype: int64

La mayor cantidad de accidentes se presentó en carreteras urbanas mientras que la menor cantidad se presentó en carreteras rurales.

In [64]:
# Describimos las variable categorica Vehicle_Type
df_accidents_categorica['Vehicle_Type'].value_counts()

bike    5338
Name: Vehicle_Type, dtype: int64

En cuanto al tipo de vehiculo, todos los accidentes se presentaron con bicicletas.

In [65]:
# Describimos las variable categorica Did_Police_Officer_Attend_Scene_of_Accident
df_accidents_categorica['Did_Police_Officer_Attend_Scene_of_Accident'].value_counts()

 1    4059
 2    1231
 3      46
-1       2
Name: Did_Police_Officer_Attend_Scene_of_Accident, dtype: int64

En cuanto a si un oficial de policia asisitio al accidente, la mayoria de los accidentes si contaron con la asistencia de un oficial de policia, mientras que la menor cantidad no contó con el reporte de un oficial de policia.